# 06. 토픽 모델링(Topic Modeling)
* 토픽=주제
* 토픽 모델링(Topic Modeling)
    * 정의: 기계 학습 및 자연어처리 분야에서 토픽이라는 문서 집합의 추상적인 주제를 발견하기 위한 통계적 모델 중 하나
    * 텍스트 본문의 숨겨진 의미 구조 발견하기 위해 사용되는 텍스트 마이닝 기법

    

## 1) 잠재 의미 분석(Latent Semantic Analysis, LSA)
* LSA: 토픽 모델링에 최적화된 알고리즘은 아니지만 토픽 모델링이라는 분야에 아이디어 제공함
    * LDA가 더 적합
* BoW 기반 DTM이나 TF-IDF: 단어의 빈도 수 이용한 수치화 방법. **단어의 이미 고려X**
    * 대안: DTM의 잠재된(Latent) 의미를 끌어내는 방법으로 LSA/LSI
        * 이 방법을 위해서는 선형대수학의 특이값 분해(Singular Value Decomposition, SVD) 이해 필요

### 1. 특이값 분해(Singular Value Decomposition, SVD)
    * 실수 벡터 공간에 한정
    * SVD: A가 mxn 행렬, 다음과 같이 3개의 행렬의 곱으로 분해
        * A=UΣVT
            * U: mxm 직교행렬, V: nxn 직교행렬, Σ: mxn 직사각 대각행렬
            * 직교행렬(orthogonal matrix): 자신과 자신의 전치행렬(transposed matrix)의 곱 또는 이를 반대로 곱한 결과가 단위행렬(identity matrix)이 되는 행렬
            * 대각행렬(diagonal matrix): 주대각선을 제외한 곳의 원소가 모두 0인 행렬
        * SVD로 나온 대각 행렬의 대각 원소의 값을 행렬 A의 특이값(singular value)

#### 1) 전치 행렬(Transposed Matrix)
* 원래의 행렬에서 행과 열을 바꾼 행렬. 반사 대칭

#### 2) 단위 행렬(Identity Matrix)
* 주대각선의 원소가 모두 1이며 나머지 원소는 모두 0인 정사각 행렬. 줄여서 대문자 I로도 표현.

#### 3) 역행렬(Inverse Matrix)
* 만약 행렬 A와 어떤 행렬을 곱했을 때, 결과로서 단위 행렬이 나온다면 이 때의 어떤 행렬을 A의 역행렬이라고 함
* A^(-1)로 표현

#### 4) 직교행렬(Orthogonal Matrix)
* **_실수 nxn 행렬 A에 대해서 A x A^T = I를 만족하면서 A^T x A = I를 만족하는 행렬_**
* **_역행렬의 정의 생각해보면, 결국 직교 행렬은 A^(-1) = A^T_**

#### 5) 대각행렬(Diagonal Matrix)
* 대각행렬(diagonal matrix)는 주대각선을 제외한 곳의 원소가 모두 0인 행렬을 말함
* 만약 직사각행렬: mxn일 때 m>n 혹은 m<n
* **_SVD를 통해 나온 대각 행렬 Σ: 대각 행렬 Σ의 주대각원소를 행렬 A의 특이값(singular value)라고 하며, 특이값은 내림차순으로 정렬되어 있다는 특징을 가짐_**

                           

### 2. 절단된 SVD(Truncated SVD)
* 위의 SVD는 full SVD
* LSA의 경우 절단된 SVD를 사용


* 대각 행렬 Σ의 대각 원소의 값 중에서 상위값 t개만 남게 됨
* 값의 손실 일어나므로 기존의 행렬 A를 복구할 수 없음
* U행렬과 V행렬의 t열까지만 남기게 됨 (t: 우리가 찾고자하는 토픽의 수를 반영한 하이퍼파라미터값)
    * 하이퍼파라미터: 사용자가 직접 값을 선택하여 성능에 영향을 주는 매개변수
        * t 크게 잡으면 기존의 행렬 A로부터 다양한 의미 가져갈 수 있음
        * t 작게 잡으면 노이즈 제거
        

* 일부 데이터 삭제-> 데이터 차원 줄인다
    * 풀 SVD보다 계산 비용 줄어들고, 상대적으로 중요하지 않은 정보 삭제(ex. 영상 처리 분야에서 노이즈 제거)
    * 자연어처리: 설명력이 낮은 정보 삭제, 설명력 높은 정보 남김. 기존의 행렬에서 드러나지 않았던 심층적인 의미 확인 가능

### 3. 잠재 의미 분석(Latent Sematic Analysis, LSA)
* LSA는 기본적으로 DTM이나 TF-IDF 행렬에 절단된 SVD(truncated SVD)를 사용하여 차원을 축소시키고, 단어들의 잠재적인 의미를 끌어낸다는 아이디어

In [1]:
import numpy as np
# 4x9 크기를 가지는 DTM 생성
A=np.array([[0,0,0,1,0,1,1,0,0],[0,0,0,1,1,0,1,0,0],[0,1,1,0,2,0,0,0,0],[1,0,0,0,0,0,0,1,1]])
np.shape(A)

(4, 9)

In [2]:
# 풀 SVD 수행. 대각 행렬의 변수면 S 사용.
# 4x4의 크기를 가지는 직교 행렬 U가 생성되었음
U, s, VT = np.linalg.svd(A, full_matrices = True)
print(U.round(2))
np.shape(U)

[[ 0.24  0.75  0.    0.62]
 [ 0.51  0.44 -0.   -0.74]
 [ 0.83 -0.49 -0.    0.27]
 [ 0.   -0.    1.   -0.  ]]


(4, 4)

In [3]:
# 대각 행렬 S
print(s.round(2))
np.shape(s)

[2.69 2.05 1.73 0.77]


(4,)

* Numpy의 linalg.svd()는 특이값 분해의 결과로 대각 행렬이 아니라 특이값의 리스트를 반환
* 앞서 본 수식의 형식으로 보려면 이를 다시 대각 행렬로 바꿔줘야 함
* 우선 특이값을 s에 저장. 대각 행렬 크기의 행렬 생성. 그 행렬에 특이값을 삽입

In [4]:
# 4 x 9 크기를 가지는 대각 행렬 S 생성. 내림차순을 보임.

S = np.zeros((4,9)) # 대각행렬의 크기인 4x9의 임의의 행렬 생성
S[:4, :4] = np.diag(s) # 특이값을 대각 행렬에 삽입
print(S.round(2))
np.shape(S)

[[2.69 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   2.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.73 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.77 0.   0.   0.   0.   0.  ]]


(4, 9)

In [5]:
# 9x9 크기를 가지는 직교 행렬 VT(V의 전치 행렬)이 생성됨
print(VT.round(2))
np.shape(VT)

[[ 0.    0.31  0.31  0.28  0.8   0.09  0.28  0.    0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]
 [ 0.58 -0.    0.    0.   -0.    0.   -0.    0.58  0.58]
 [-0.    0.35  0.35 -0.16 -0.25  0.8  -0.16  0.    0.  ]
 [-0.   -0.78 -0.01 -0.2   0.4   0.4  -0.2   0.    0.  ]
 [-0.29  0.31 -0.78 -0.24  0.23  0.23  0.01  0.14  0.14]
 [-0.29 -0.1   0.26 -0.59 -0.08 -0.08  0.66  0.14  0.14]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19  0.75 -0.25]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19 -0.25  0.75]]


(9, 9)

In [6]:
# U x S x VT를 하면 기존의 행렬 A가 나와야 함
# Numpy의 allclose()는 2개의 행렬이 동일하면 True를 리턴. 
# 이를 사용하여 정말로 기존의 행렬 A와 동일한지 확인해봄

np.allclose(A, np.dot(np.dot(U, S), VT).round(2))

True

In [8]:
# 지금까지는 풀 SVD
# 이제 t를 정하고, 절단된 SVD 수행. t = 2

# 대각행렬 S 내의 특이값 중에서 상위 2개만 남기고 제거
S = S[:2, :2]
print(S.round(2))

[[2.69 0.  ]
 [0.   2.05]]


In [9]:
# 직교행렬 U
U = U[:, :2] # 2개의 열만 남기고 제거
print(U.round(2))

[[ 0.24  0.75]
 [ 0.51  0.44]
 [ 0.83 -0.49]
 [ 0.   -0.  ]]


In [10]:
# 전치행렬인 VT에 대해서 2개의 행만 남기고 제거
VT = VT[:2, :]
print(VT.round(2))

[[ 0.    0.31  0.31  0.28  0.8   0.09  0.28  0.    0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]]


In [12]:
# 축소된 행렬 U, S, VT에 대해서 다시 U x S x VT 연산을 하면 기존 A와는 다른 결과가 나옴
# 값이 손실되었으므로 이 세 개의 행렬로는 이제 기존의 A 행렬 복구 불가능
A_prime = np.dot(np.dot(U, S), VT)
print(A)
print(A_prime.round(2))

# 0인 값들은 0에 가까운 값. 1인 값들은 1에 가까운 값.
# 값이 제대로 복구되지 않은 구간도 보임

[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
[[ 0.   -0.17 -0.17  1.08  0.12  0.62  1.08 -0.   -0.  ]
 [ 0.    0.2   0.2   0.91  0.86  0.45  0.91  0.    0.  ]
 [ 0.    0.93  0.93  0.03  2.05 -0.17  0.03  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.  ]]


차원 축소된 U, S, VT의 크기가 어떤 의미를 가지고 있는지
* **축소된 U**: 4x2의 크기. 문서의 개수 x 토픽의 수 t의 크기.
    * 단어의 개수인 9는 유지되지 않는데 문서의 개수인 4의 크기가 유지 되었으니 4개의 문서 각각을 2개의 값으로 표현하고 잇음
    * U의 각 행은 _잠재 의미를 표현하기 위한 수치화된 각각의 **문서** 벡터_
    
    
* **축소된 VT**: 2x9의 크기. 토픽의 수 t x 단어의 개수의 크기
    * VT의 각 열은 _잠재 의미를 표현하기 위해 수치화된 각각의 **단어** 벡터_
    
    
* 이 문서와 단어 벡터들을 통해 다른 문서의 유사도, 다른 단어의 유사도, 단어(쿼리)로부터 문서의 유사도를 구하는 것이 가능해짐

### 4. 실습을 통한 이해
* 사잇킷런의 Twenty Newsgroups라고 불리는 20개의 다른 주제를 가진 뉴스 그룹 데이터 제공
* LSA 사용해서 문서의 수를 원하는 토픽의 수로 압축한 뒤에 각 토픽 당 가장 중요한 단어 5개를 출력하는 실습으로 토픽 모델링 수행
* 뉴스그룹 데이터는 뉴스 데이터가 아님

#### 1) 뉴스 그룹 데이터에 대한 이해

In [21]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

11314

In [ ]:
documents[1]
# 특수문자가 포함된 다수의 영어 문장으로 구성되어 있음
# 샘플이 총 11314개 존재

In [ ]:
# 사이킷런이 존재하는 어떤 20개의 카테고리를 갖고있었는지 저장되어 있음.
print(dataset.target_names)

#### 2) 텍스트 전처리
* 알파벳을 제외한 구두점, 숫자, 특수 문자 제거
* 정규표현식, 짧은 단어 제거, 알파벳 모두 소문자로 하여 단어의 개수 줄임

In [ ]:
# 데이터 정제
news_df = pd.DataFrame({'document': document})
# 특수문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3 이하인 단어는 제거
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [ ]:
news_df['clean_doc'][1]

In [ ]:
# 뉴스그룹 데이터에서 불용어 제거. 토큰화 우선 수행 후 불용어 제거.
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [ ]:
print(tokenized_doc[1])

#### 3) TF-IDF 행렬 만들기
* TfidfVectorizer를 사용해서 TF-IDF 행렬을 만들기 위해서는 다시 토큰화 작업을 역으로 취소하는 작업을 수행
* **역토큰화(Detokenization)**

In [ ]:
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
    
news_df['clean_doc'] = detokenized_doc

In [ ]:
news_df['clean_doc'][1]

In [17]:
# 사잇킷런의 TfidfVectorizer를 통해 단어 1000개에 대한 TF-IDF 행렬 만듦
from sklearn.feature_extraction.text import TfidfVectorizer

# 상위 1000개의 단어 보존
vectorizer = TfidfVectorizer(stop_words='english', 
                             max_features=1000, max_df=0.5, smooth_idf=True)
X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape

NameError: name 'news_df' is not defined

#### 4) 토픽 모델링
* TF-IDF 행렬을 다수의 행렬로 분해
* Truncated SVD 사용 for 차원 축소
* 20개의 토픽 가졌다고 가정
* 토픽의 숫자는 n_components 파라미터로 지정 가능

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', 
                         n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

In [ ]:
np.shape(svd_model.components_) # LSA에서 VT
# 토픽의 수 t x 단어의 수의 크기

In [ ]:
terms = vectorizer.get_feature_names() # 단어 집합. 1000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" %(idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n -1:-1]])
get_topics(svd_model.components_, terms)

# 각 20개의 행의 각 1000개의 열 중 가장 값이 큰 5개의 값을 찾아서 단어로 출력

### 5. LSA의 장단점
* 쉽고 빠르게 구현 가능. 단어의 잠재적 의미 이끌어낼 수 있어 문서의 유사도 계산 등에서 좋은 성능
* SVD 특성상 이미 계산된 LSA에 새로운 데이터를 추가하여 계산하려고 하면 보통 처음부터 다시 계산해야 함
    * 새로운 정보에 대해 업데이트 어려움
* 최근에는 LSA 대신 Word2Vec 등 단어의 의미를 벡터화할 수 있는 인공 신경망 기반의 방법론이 각광받는 이유